# BayesFrag - Tutorial 3: Computation of GMM estimates using OpenQuake

In [1]:
import numpy as np
import pandas as pd
import os
import json
import openquake.hazardlib as oq

In [2]:
path_data = os.getcwd() + os.sep + 'data_twodim' + os.sep

### Rupture information

The rupture metadata is from ESM: https://esm-db.eu/#/event/IT-2009-0009 

The rupture geometry is from INGV: http://shakemap.ingv.it/shake4/downloadPage.html?eventid=1895389 
The latter is identical to the ESM rupture geometry!

In [3]:
Point = oq.geo.point.Point
PlanarSurface = oq.geo.surface.planar.PlanarSurface
MultiSurface = oq.geo.surface.multi.MultiSurface
BaseRupture = oq.source.rupture.BaseRupture

f = open(path_data + 'rupture.json')
rup_temp = json.load(f)
f.close()
rup_geom_json = rup_temp['features'][0]['geometry']
rup_geom = np.array(rup_geom_json['coordinates'][0][0])[:-1,:]

rupture_surface = PlanarSurface.from_corner_points(
    top_left = Point(rup_geom[0, 0], rup_geom[0, 1], rup_geom[0, 2]),
    top_right = Point(rup_geom[1, 0], rup_geom[1, 1], rup_geom[1, 2]),
    bottom_right = Point(rup_geom[2, 0], rup_geom[2, 1], rup_geom[2, 2]),
    bottom_left = Point(rup_geom[3, 0], rup_geom[3, 1], rup_geom[3, 2]),
)
rupture = BaseRupture(mag = 6.1, rake = -90.0, 
                    tectonic_region_type = 'Active Shallow Crust', 
                    hypocenter = Point(longitude = 13.380, 
                                        latitude = 42.342,
                                        depth = 8.3),
                    surface = rupture_surface)

### Sites information

In [4]:
# Raw site information for station sites and building survey sites
dfstations = pd.read_csv(path_data + 'stations.csv')
dfsurvey = pd.read_csv(path_data + 'survey2.csv')

### Compute GMM estimates

In [10]:
args = {
    'im_string': 'SAT0_300',
    'GMM': 'BindiEtAl2011',
        }

In [11]:
if args['GMM'] == 'BindiEtAl2011':
    gmm = oq.gsim.bindi_2011.BindiEtAl2011()
    # This GMM is defined for geometric mean IM 
    # (also called average horizontal)
    obs_str = 'geoM_log' + args['im_string']
elif args['GMM'] == 'ChiouYoungs2014Italy':
    gmm = oq.gsim.chiou_youngs_2014.ChiouYoungs2014Italy()
    # This GMM is defined for RotD50
    obs_str = 'rotD50_log' + args['im_string']

if args['im_string'] == 'PGA':
    im_list = [oq.imt.PGA()]
else:
    T = float('.'.join( args['im_string'][3:].split('_') )) 
    im_list = [oq.imt.SA(T)]

In [12]:
Mesh = oq.geo.mesh.Mesh
def get_EpicentralAzimuth(rupture, sites_lons, sites_lats):
    sites_mesh = Mesh(sites_lons, 
                    sites_lats, 
                    depths=None)
    return rupture.surface.get_azimuth(sites_mesh)

def get_RuptureContext(rupture, sites_lons, sites_lats, sites_vs30, 
                sites_vs30measured=None, sites_z1pt0=None):
    sites_mesh = Mesh(sites_lons, 
                      sites_lats, 
                      depths=None)
    rctx = oq.contexts.RuptureContext()
    rctx.rjb = rupture.surface.get_joyner_boore_distance(sites_mesh)
    rctx.rrup = rupture.surface.get_min_distance(sites_mesh)
    rctx.vs30 = sites_vs30
    rctx.mag = rupture.mag * np.ones_like(rctx.rjb)
    rctx.rake = rupture.rake * np.ones_like(rctx.rjb)
    rctx.rx = rupture.surface.get_rx_distance(sites_mesh)
    rctx.ztor = rupture.surface.get_top_edge_depth() * np.ones_like(rctx.rjb)
    rctx.dip = rupture.surface.get_dip() * np.ones_like(rctx.rjb)
    if sites_z1pt0 is None:
        rctx.z1pt0 = -7.15/4 * np.log( (sites_vs30**4 + 571**4) / (1360**4 + 571**4) )
    else: 
        rctx.z1pt0 = sites_z1pt0
    if sites_vs30measured is None:
        rctx.vs30measured = False
    else:
        rctx.vs30measured = sites_vs30measured
    return rctx    

def compute_GMM(gmm, im_list, rupture_context):
    n = len(rupture_context.vs30)
    nim = len(im_list)
    mean = np.zeros([nim, n])
    sigma = np.zeros([nim, n])
    tau = np.zeros([nim, n])
    phi = np.zeros([nim, n])
    gmm.compute(rupture_context, im_list, mean, sigma, tau, phi)
    return {'mu_logIM': mean, 
            'tau_logIM': tau, 
            'phi_logIM': phi}

In [13]:
df = dfstations.copy()

# Epicentral azimuth may be required for some spatial correlation models
epicentral_azimuth = get_EpicentralAzimuth(rupture, df['Longitude'].values, df['Latitude'].values)
df['epiazimuth'] = epicentral_azimuth.squeeze()

# Compute GMM estimates
rupture_context = get_RuptureContext(rupture, df['Longitude'].values, df['Latitude'].values, 
                                        df['vs30'].values, df['vs30measured'].values)
res_GMM = compute_GMM(gmm, im_list, rupture_context)
for key in res_GMM.keys():
    df[key] = res_GMM[key].squeeze()

# Extract Observed IM
df['obs_logIM'] = df[obs_str].values

# Arrange final csv file to be used for fragility function estimation
dfstations = df[['network_code', 'station_code', 'Longitude', 'Latitude', 'vs30', 
                'epiazimuth', 'mu_logIM', 'tau_logIM', 'phi_logIM', 'obs_logIM']].copy()
dfstations.to_csv(path_data + 'stations_im_' + args['im_string'] + '_GMM_' + args['GMM'] + '.csv', index=False)

In [13]:
df = dfsurvey.copy()

# Epicentral azimuth may be required for some spatial correlation models
epicentral_azimuth = get_EpicentralAzimuth(rupture, df['Longitude'].values, df['Latitude'].values)
df['epiazimuth'] = epicentral_azimuth.squeeze()

# Compute GMM estimates
rupture_context = get_RuptureContext(rupture, df['Longitude'].values, df['Latitude'].values, 
                                        df['vs30'].values)
res_GMM = compute_GMM(gmm, im_list, rupture_context)
for key in res_GMM.keys():
    df[key] = res_GMM[key].squeeze()

# # Arrange final csv file to be used for fragility function estimation
# dfsurvey = df[['id', 'Longitude', 'Latitude', 'vs30', 
#                 'epiazimuth', 'mu_logIM', 'tau_logIM', 'phi_logIM', 
#                 'BuildingClass', 'DamageState']].copy()
# dfsurvey.to_csv(path_data + 'survey2_im_' + args['im_string'] + '_GMM_' + args['GMM'] + '.csv', index=False)

In [15]:
dfsurvey = df[['id', 'Longitude', 'Latitude', 'vs30', 'BuildingClass',
                'epiazimuth', 'mu_logIM', 'tau_logIM', 'phi_logIM', 
                 ]].copy()
dfsurvey.to_csv(path_data + 'survey2_im_' + args['im_string'] + '_GMM_' + args['GMM'] + '.csv', index=False)

In [16]:
dfsurvey

,id,Longitude,Latitude,vs30,BuildingClass,epiazimuth,mu_logIM,tau_logIM,phi_logIM
0,0,13.430660,42.302485,529.93370,A,44.626553,-0.565073,0.501964,0.66775
1,1,13.288765,42.396429,509.20395,A,164.165307,-0.677659,0.501964,0.66775
2,2,13.365190,42.365922,450.50436,A,169.837767,-0.565073,0.501964,0.66775
3,3,13.402773,42.348720,453.86163,A,177.774937,-0.565073,0.501964,0.66775
4,4,13.400559,42.352410,455.24070,A,179.724174,-0.565073,0.501964,0.66775
...,...,...,...,...,...,...,...,...,...
4195,4195,13.114247,42.017778,434.47626,C,77.688827,-2.456111,0.501964,0.66775
4196,4196,13.806123,42.479776,521.24426,C,280.317427,-2.181034,0.501964,0.66775
4197,4197,13.301170,42.288853,462.58060,C,110.153346,-0.821560,0.501964,0.66775
4198,4198,13.428467,42.028974,392.25024,C,40.589708,-1.844378,0.501964,0.66775


In [38]:
# data_dir = os.path.join('..', '..', 'data_aquila', "")
# dfsm = pd.read_csv(data_dir + 'vs30_shakemap.csv')
# mask = ((dfsm.Longitude >= 13.2) & (dfsm.Longitude <= 13.5) &
#         (dfsm.Latitude >= 42.15) & (dfsm.Latitude <= 42.5))
# df = dfsm[mask].copy()
# rows = df.row.unique()
# cols = df.col.unique()
# df = dfsm[np.isin(dfsm.row.values,rows)&np.isin(dfsm.col.values,cols)].copy()
# sh = (len(df.row.unique()), len(df.col.unique()))
# X = df.Longitude.values.reshape(sh)
# Y = df.Latitude.values.reshape(sh)
# Z = df.vs30.values.reshape(sh)
# df[['row', 'col', 'Longitude', 'Latitude', 'vs30']].to_csv(path_data + 'gridmap.csv', index=False)

In [42]:
df = pd.read_csv(path_data + 'gridmap.csv')

In [44]:
df = pd.read_csv(path_data + 'gridmap.csv')

# Epicentral azimuth may be required for some spatial correlation models
epicentral_azimuth = get_EpicentralAzimuth(rupture, df['Longitude'].values, df['Latitude'].values)
df['epiazimuth'] = epicentral_azimuth.squeeze()

# Compute GMM estimates
rupture_context = get_RuptureContext(rupture, df['Longitude'].values, df['Latitude'].values, 
                                        df['vs30'].values)
res_GMM = compute_GMM(gmm, im_list, rupture_context)
for key in res_GMM.keys():
    df[key] = res_GMM[key].squeeze()

# Arrange final csv file to be used for fragility function estimation
dfsurvey = df[['row', 'col', 'Longitude', 'Latitude', 'vs30', 
                'epiazimuth', 'mu_logIM', 'tau_logIM', 'phi_logIM']].copy()
dfsurvey.to_csv(path_data + 'gridmap_im' + im_string + '_' + gmm_combo + '.csv', index=False)

In [58]:
df = pd.read_csv('data_onedim/OneDim_buildings.csv')

In [59]:
df = df.rename(columns={
    'mu_logPGA': 'mu_logIM',
    'tau_logPGA': 'tau_logIM',
    'phi_logPGA': 'phi_logIM',
    'rec_logPGA': 'sim_logIM',
    'VulClass': 'BuildingClass',
    'obs_DS': 'DamageState'
})

In [60]:
mask = (df.DamageState >= 3)
df.loc[df[mask].index, 'DamageState'] = int(2)
df['BuildingClass'] = 'A'

In [62]:
df.to_csv('data_onedim/survey_withIM_PGA.csv', index=False)

In [54]:
# Extract two indices as seismic network stations
idx_SS = [61, 362]
dfstations = df.loc[idx_SS].copy()
dfstations

,x,y,vs30,mu_logIM,tau_logIM,phi_logIM,sim_logIM,BuildingClass,DamageState
61,-15.110220,0,450,-1.891671,0.3581,0.6375,-2.674296,0,0
362,9.018036,0,450,-1.377212,0.3581,0.6375,-1.744287,0,0


In [57]:
dfstations.rename(columns={'sim_logIM': 'obs_logIM'}).drop(columns=['BuildingClass', 'DamageState']).to_csv('data_onedim/stations_withIM_PGA.csv', index=False)